<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


# 1. Calling openai using requests package

In [17]:
import os 
from dotenv import load_dotenv 

load_dotenv(override=True)

api_key=os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith("sk-proj"): 
    pass
else: 
    print("OpenAI key not found")

In [4]:
import requests 
headers={
    "Authorization" : f"Bearer {api_key}",
    "Content-Type" : "application/json"
}

payload={
    "model": "gpt-5-nano",
    "messages" : [
        {"role" : "user", "content": "Tell me a fun fact"}
    ]
}
payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [5]:
# make request to open ai 
response=requests.post(
    "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
)
response.json()

{'id': 'chatcmpl-CUZDfnHNfs2sLha0uvNRtQaAmJfqI',
 'object': 'chat.completion',
 'created': 1761401259,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: Bananas are technically berries, but strawberries aren’t. Botanically, a berry comes from a single flower’s ovary, and bananas fit that definition, while strawberries don’t. Want another fun fact?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 1013,
  'total_tokens': 1024,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 960,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [8]:
# to find the content we are interested in we need to 
response.json()["choices"][0]["message"]["content"]

'Fun fact: Bananas are technically berries, but strawberries aren’t. Botanically, a berry comes from a single flower’s ovary, and bananas fit that definition, while strawberries don’t. Want another fun fact?'

# Using OpenAI package - To call OpenAI 
``` 
Also called Python Client Library. It is just a wrapper around to make call to http endpoint. Helps to work in Pythonic way instead of messy json. 

In [9]:
from openai import OpenAI

messages=[{"role" : "user", "content": "Tell me a fun fact"}]

# create openai object - no need to explicitly provide api key. It will by default pick from .env file 
open_ai=OpenAI()
response=open_ai.chat.completions.create(
    model="gpt-5-nano", 
    messages=messages
)
response.choices[0].message.content

'Fun fact: A day on Venus is longer than its year. It takes about 243 Earth days to rotate once, but only about 225 Earth days to orbit the Sun.'

# Using OpenAI package - To call Gemini (Google's model)

In [11]:
gemini_api_key=os.getenv("GOOGLE_API_KEY")
gemini_base_url=os.getenv("GEMINI_BASE_URL")

gemini=OpenAI(base_url=gemini_base_url, api_key=gemini_api_key)
response=gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
response.choices[0].message.content

"Here's one!\n\nA group of **owls** is called a **parliament**."

# Using OpenAI package - To call Local ollama models like Llama3.1

In [18]:
ollama_api_key="ollama"
ollama_base_url=os.getenv("OLLAMA_BASE_URL")

ollama=OpenAI(base_url=ollama_base_url, api_key=ollama_api_key)
response=ollama.chat.completions.create(model="llama3.1", messages=messages)
response.choices[0].message.content

'Here\'s one:\n\n**There is a species of jellyfish that is immortal!**\n\nThe Turritopsis dohrnii, also known as the "immortal jellyfish," is a type of jellyfish that can transform its body into a younger state through a process called transdifferentiation. This means that it can essentially revert back to its polyp stage, which is the juvenile form of a jellyfish, and then grow back into an adult again. This process can be repeated indefinitely, making Turritopsis dohrnii theoretically immortal!\n\nIsn\'t that cool?'

# Day 1 excercise as assignment using ollama models 

``` 
Read the content from the website using scraper program provided in headeless manner and ask agent to summarize in most snarky way 

In [19]:
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [20]:
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [21]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": user_prompt_prefix + website}
    ]


In [22]:
from scraper import fetch_website_contents

# call open ai 
def summarize(url):
    website=fetch_website_contents(url)
    ollama=OpenAI(base_url=ollama_base_url, api_key=ollama_api_key)
    response=ollama.chat.completions.create(model="llama3.1", messages=messages_for(website))
    return response.choices[0].message.content

In [24]:
from IPython.display import Markdown, display
display(Markdown(summarize("https://edwarddonner.com")))

**A website by Edward Donner, because the world wasn't already sufficiently overrun with code and tech startups**

This website appears to be a personal blog and portfolio by Edward Donner, who is obsessed with AI and LLMs (Large Language Models). He's a co-founder of Nebula.io, some company that uses AI to help recruiters find talent, and claims it's patented.

**Recent Posts:**

* He wrote about the awesomeness of something called Agentic AI on AWS (whatever that is)
* He wants to connect with you, because who doesn't love a good personal connection
* There are some courses and briefings, probably full of buzzwords like "LLM expert" and "leader"

**Outsmart:** An arena where LLMs battle each other in a game of diplomacy and deviousness. Think virtual Robot Wars, but with code.